In [1]:
import os
import glob
import pandas as pd
from tqdm import tqdm
from sacrebleu.metrics import BLEU
import json


def match_model_name(filename):
    if "Qwen2.5-1.5B-Instruct" in filename:
        model_name = "Qwen2.5-1.5B-Instruct"
    elif "Phi-3.5-mini-instruct" in filename:
        model_name = "Phi-3.5-mini-instruct"
    elif "Llama-3.2-3B-Instruct" in filename:
        model_name = "Llama-3.2-3B-Instruct"
    elif "Qwen2.5-3B-Instruct" in filename:
        model_name = "Qwen2.5-3B-Instruct"
    elif "Llama-3.2-1B-Instruct" in filename:
        model_name = "Llama-3.2-1B-Instruct"
    elif "Llama-2-7b-chat-hf" in filename:
        model_name = "Llama-2-7b-chat-hf"
    elif "Llama-2-13b-chat-hf" in filename:
        model_name = "Llama-2-13b-chat-hf"
    elif "Llama-2-70b-chat-hf" in filename:
        model_name = "Llama-2-70b-chat-hf"
    elif "Meta-Llama-3-8B-Instruct" in filename:
        model_name = "Meta-Llama-3-8B-Instruct"
    elif "Meta-Llama-3-70B-Instruct" in filename:
        model_name = "Meta-Llama-3-70B-Instruct"
    elif "Mistral-7B-Instruct-v0.2" in filename:
        model_name = "Mistral-7B-Instruct-v0.2"
    elif "Mixtral-8x7B-Instruct-v0.1" in filename:
        model_name = "Mixtral-8x7B-Instruct-v0.1"
    elif "DeepSeek-R1-Distill-Llama-70B" in filename:
        model_name = "DeepSeek-R1-Distill-Llama-70B"
    elif "DeepSeek-R1-Distill-Qwen-1.5B" in filename:
        model_name = "DeepSeek-R1-Distill-Qwen-1.5B"
    elif "Llama-3.1-8B-Instruct" in filename:
        model_name = "Llama-3.1-8B-Instruct"
    else:
        raise ValueError(f"Unknown model name in filename: {filename}")
    return model_name


def compute_bleu_score(row):
    generated_response = row["generate_response"]
    reference_answer = row["target"]

    if not generated_response:  # Handles "", None, etc.
        return 0.0

    score = bleu.sentence_score(
        hypothesis=generated_response, references=[reference_answer]
    ).score
    return score


# Setup
input_root_folder = "/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling"
evaluator_model_name = "Llama-3.3-70B-Instruct"
output_folder = input_root_folder
bleu = BLEU(tokenize="flores101", effective_order=True)
tqdm.pandas()

# ✅ Find all .jsonl files directly inside the folder
jsonl_files = glob.glob(os.path.join(input_root_folder, "*.jsonl"))

print(f"Found {len(jsonl_files)} .jsonl files in {input_root_folder}")

# Process each .jsonl file
for file_path in jsonl_files:
    try:
        print(f"\nProcessing: {file_path}")
        with open(file_path, "r") as file:
            print(file_path)
            records = []
            for line in file:
                line = line.strip()
                if not line:
                    continue
                try:
                    record = json.loads(line)
                    records.append(record)
                except json.JSONDecodeError as e:
                    print(f"[WARNING] Skipping bad line in {file_path}: {e}")
            df = pd.DataFrame(records)

            df["model_name"] = match_model_name(file_path)

        MT_df = df[df["category"] == "MT"]
        if MT_df.empty:
            print(f"  No 'MT' entries in {file_path}. Skipping.")
            continue

        MT_df["MT_accuracy"] = MT_df.progress_apply(compute_bleu_score, axis=1)

        model_name = os.path.splitext(os.path.basename(file_path))[0]
        output_prefix = f"evaluated_{evaluator_model_name}_{model_name}"
        output_path = os.path.join(output_folder, f"{output_prefix}_MT.jsonl")

        MT_df.to_json(output_path, lines=True, orient="records")
        print(f"  Saved to: {output_path}")

    except Exception as e:
        print(f"  Error processing {file_path}: {e}")

Found 81 .jsonl files in /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling

Processing: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_194310_r_2.0_p_0.8_k_2.jsonl
/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_194310_r_2.0_p_0.8_k_2.jsonl
[WARNING] Skipping bad line in /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_194310_r_2.0_p_0.8_k_2.jsonl: Unterminated string starting at: line 1 column 2675 (char 2674)


100%|██████████| 2100/2100 [00:00<00:00, 4931.38it/s]
/tmp/ipykernel_3440479/1085042317.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MT_df["MT_accuracy"] = MT_df.progress_apply(compute_bleu_score, axis=1)


  Saved to: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/evaluated_Llama-3.3-70B-Instruct_vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_194310_r_2.0_p_0.8_k_2_MT.jsonl

Processing: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Mixtral-8x7B-Instruct-v0.1-awq__20250412_030049_r_0.0_p_1.0_k_10.jsonl
/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Mixtral-8x7B-Instruct-v0.1-awq__20250412_030049_r_0.0_p_1.0_k_10.jsonl
[WARNING] Skipping bad line in /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Mixtral-8x7B-Instruct-v0.1-awq__20250412_030049_r_0.0_p_1.0_k_10.jsonl: Unterminated string starting at: line 1 column 10 (char 9)


100%|██████████| 1800/1800 [00:00<00:00, 2439.15it/s]
/tmp/ipykernel_3440479/1085042317.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MT_df["MT_accuracy"] = MT_df.progress_apply(compute_bleu_score, axis=1)


  Saved to: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/evaluated_Llama-3.3-70B-Instruct_vllm_exp_dataset_csv_Mixtral-8x7B-Instruct-v0.1-awq__20250412_030049_r_0.0_p_1.0_k_10_MT.jsonl

Processing: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_202955_r_0.0_p_0.9_k_2.jsonl
/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_202955_r_0.0_p_0.9_k_2.jsonl
[WARNING] Skipping bad line in /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250407_202955_r_0.0_p_0.9_k_2.jsonl: Unterminated string starting at: line 1 column 10 (char 9)


...

100%|██████████| 1800/1800 [00:00<00:00, 4495.94it/s]
/tmp/ipykernel_3440479/1085042317.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MT_df["MT_accuracy"] = MT_df.progress_apply(compute_bleu_score, axis=1)


  Saved to: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/evaluated_Llama-3.3-70B-Instruct_vllm_exp_dataset_csv_Llama-3.1-8B-Instruct-awq__20250409_175852_r_2.0_p_0.9_k_10_MT.jsonl

Processing: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250408_043304_r_0.0_p_0.9_k_5.jsonl
/home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250408_043304_r_0.0_p_0.9_k_5.jsonl
[WARNING] Skipping bad line in /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250408_043304_r_0.0_p_0.9_k_5.jsonl: Unterminated string starting at: line 1 column 10 (char 9)


100%|██████████| 1800/1800 [00:00<00:00, 5187.93it/s]

  Saved to: /home/snt/projects_lujun/temperature_eval_github/temperature_eval/data/Additional_Results/new_topkp_sampling/evaluated_Llama-3.3-70B-Instruct_vllm_exp_dataset_csv_Llama-3.2-1B-Instruct-awq__20250408_043304_r_0.0_p_0.9_k_5_MT.jsonl



/tmp/ipykernel_3440479/1085042317.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MT_df["MT_accuracy"] = MT_df.progress_apply(compute_bleu_score, axis=1)
